## FYI Cohere Aya23 Model API

- Model Name : `c4ai-aya-23`
- Rate Limit : `10` (Trial Key) and `10k` (Production Key) Requests Per Minute. [Rate Limit Page](https://docs.cohere.com/docs/rate-limits#trial-key-limitations)
- API Credits: There is no info about pricing for aya model. [Pricing Page](https://cohere.com/pricing)
- API Documentation: [/chat](https://docs.cohere.com/reference/chat)

### Data Exploration

- `id` is same in `blip_laion_cc_sbu_558k.json` and `blip_laion_cc_sbu_558k_meta.json`
- `blip_caption` in the metadata file relates to original file as follows
  ```python
      for conversation in row['conversations']:
          if conversation['from'] == 'gpt':
              blip_caption = conversation['value']
  ```
- Sample Conversation:
  ```json
    [
        {
            'from': 'human',
            'value': 'Share a concise interpretation of the image provided.\n<image>'
        },
        {
            'from': 'gpt',
            'value': 'the new security team is ready to take on the competition'
        }
    ]
  ```
- All the conversations are single-turn conversations in the dataset.
- In total there 22 different set of human instructions in the dataset in which either `\n<image>` is added as suffix or `<image>\n` is added as prefix.
- If we remove the prefix and suffix, there are only 11 different instructions which we need to translate.

In [18]:
import sys
import pandas as pd
import time
import cupy as cp
import json

sys.path.append("../")

In [8]:
start_time = time.time()
data_df = pd.read_json("../data/blip_laion_cc_sbu_100.json")

print(time.time() - start_time)

0.0039997100830078125


In [9]:
from utils.helpers import load_json

In [12]:
data = load_json("../data/blip_laion_cc_sbu_100.json")
batch_size = 10
total_count = len(data)

In [13]:
for i in range(0, total_count, batch_size):
    batch = data[i:i + batch_size]

In [19]:
conversations = [item['conversations'] for item in batch]
conv_lengths = cp.array([len(json.dumps(conv)) for conv in conversations])
sorted_indices = cp.argsort(conv_lengths)
sorted_batch = [batch[i] for i in sorted_indices.get()]

In [21]:
unpacked_batch = []
for convo in unpacked_batch:
    info = {
        'id'

[{'id': '004271632',
  'image': '00427/004271632.jpg',
  'conversations': [{'from': 'human',
    'value': 'Give a brief description of the image.\n<image>'},
   {'from': 'gpt', 'value': 'blue cartoon fish photo greeting card'}]},
 {'id': '004330598',
  'image': '00433/004330598.jpg',
  'conversations': [{'from': 'human',
    'value': 'Give a brief description of the image.\n<image>'},
   {'from': 'gpt', 'value': 'fundamentals of finite element analysis'}]},
 {'id': '002846089',
  'image': '00284/002846089.jpg',
  'conversations': [{'from': 'human',
    'value': 'Render a clear and concise summary of the photo.\n<image>'},
   {'from': 'gpt', 'value': 'an image of vintage reindeer sweater'}]},
 {'id': '003174711',
  'image': '00317/003174711.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat is in the photo?'},
   {'from': 'gpt',
    'value': '1 month full page seo traffic from seo for the most profitable website'}]},
 {'id': '001377789',
  'image': '00137/001377789.

In [20]:
conversations

[[{'from': 'human',
   'value': 'Render a clear and concise summary of the photo.\n<image>'},
  {'from': 'gpt', 'value': 'an image of vintage reindeer sweater'}],
 [{'from': 'human', 'value': '<image>\nDescribe the image concisely.'},
  {'from': 'gpt',
   'value': 'the quote on positiveness is one of the most difficult things people take different roads'}],
 [{'from': 'human',
   'value': 'Give a brief description of the image.\n<image>'},
  {'from': 'gpt',
   'value': "england women's cricketers batsman person plays the ball as she hits out during the cricket international match against"}],
 [{'from': 'human',
   'value': 'Give a brief description of the image.\n<image>'},
  {'from': 'gpt', 'value': 'fundamentals of finite element analysis'}],
 [{'from': 'human',
   'value': '<image>\nRender a clear and concise summary of the photo.'},
  {'from': 'gpt',
   'value': 'cut paper on a piece of craft paper and glue to create the bow'}],
 [{'from': 'human',
   'value': 'Give a brief descrip

In [3]:
metadata_df = pd.read_json("../data/blip_laion_cc_sbu_558k_meta.json")

In [4]:
data_df.head()

,id,image,conversations
0,4539375,00453/004539375.jpg,"[{'from': 'human', 'value': 'Render a clear an..."
1,2239345,00223/002239345.jpg,"[{'from': 'human', 'value': 'Write a terse but..."
2,5947502,00594/005947502.jpg,"[{'from': 'human', 'value': '<image> What is t..."
3,5116462,00511/005116462.jpg,"[{'from': 'human', 'value': '<image> Render a ..."
4,2017886,00201/002017886.jpg,"[{'from': 'human', 'value': 'What is in the ph..."


In [5]:
metadata_df.head()

,id,image,blip_caption,url
0,4539375,00453/004539375.jpg,select luxury furniture 3 - inch gel memory fo...,http://ec1.ostkcdn.com/images/products/8111140...
1,2239345,00223/002239345.jpg,a grey watch with an army style strap,https://ak1.ostkcdn.com/images/products/119322...
2,5947502,00594/005947502.jpg,a dragon kite flying in the blue sky stock images,https://thumbs.dreamstime.com/b/fliegen-dragon...
3,5116462,00511/005116462.jpg,$ 10 - cute cheap printed mini dress - khaki m...,https://media.shopscover.com/media/product/sm/...
4,2017886,00201/002017886.jpg,augmented reality using aruco markers in opencv,https://www.learnopencv.com/wp-content/uploads...


In [6]:
different_gpt_response_and_blip_caption = []
multi_turn_conversations = []
human_instructions_with_suffix_prefix = []
human_instructions_without_suffix_prefix = []

for index, row in data_df.iterrows():
    conversations = row['conversations']
    if len(conversations) > 2:
        multi_turn_conversations.append(row['id'])
        
    for conversation in row['conversations']:
        if conversation['from'] == 'human':
            instruction = conversation['value'] 
            processed_instruction = instruction.replace("\n<image>", "").replace("<image>\n", "")
            human_instructions_with_suffix_prefix.append(instruction)
            human_instructions_without_suffix_prefix.append(processed_instruction)

        if conversation['from'] == 'gpt' and conversation['value'] != metadata_df.iloc[index]['blip_caption']:
            different_gpt_response_and_blip_caption.append(row['id'])


print(f"No. of data rows which has different gpt response than the blip caption in the metadata = {len(different_gpt_response_and_blip_caption)}")
print(f"No. of Multi-turn Conversations = {len(multi_turn_conversations)}")
print(f"No. of Distinct Human Instructions With Suffix and Prefix = {len(set(human_instructions_with_suffix_prefix))}")
print(f"No. of Distinct Human Instructions Without Suffix and Prefix {len(set(human_instructions_without_suffix_prefix))}")

No. of data rows which has different gpt response than the blip caption in the metadata = 0
No. of Multi-turn Conversations = 0
No. of Distinct Human Instructions With Suffix and Prefix = 22
No. of Distinct Human Instructions Without Suffix and Prefix 11


In [8]:
set(human_instructions_without_suffix_prefix)

{'Describe the image concisely.',
 'Give a brief description of the image.',
 'Give a short and clear explanation of the subsequent image.',
 "Present a compact description of the photo's key features.",
 'Provide a brief description of the given image.',
 'Render a clear and concise summary of the photo.',
 'Share a concise interpretation of the image provided.',
 'Summarize the visual content of the image.',
 'What is in the photo?',
 'What is this?',
 'Write a terse but informative summary of the picture.'}